In [6]:
import cv2
from pyzbar.pyzbar import decode
import numpy as np
import requests

# Set the IP address of the ESP32
ESP32_IP = '192.168.213.154'
ESP32_PORT = '80'  # Default HTTP port
ESP32_URL = f'http://{ESP32_IP}:{ESP32_PORT}/detect'  # Corrected URL endpoint

# Initialize the webcam (0 is usually the built-in camera)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

print("Press 'q' to exit")

# Create simple headshot symbols for friend and enemy
headshot_size = 30
happy_face = np.zeros((headshot_size, headshot_size, 4), dtype=np.uint8)
cv2.circle(happy_face, (headshot_size // 2, headshot_size // 2), headshot_size // 2, (0, 255, 0, 255), 2)
cv2.line(happy_face, (0, headshot_size // 2), (headshot_size, headshot_size // 2), (0, 255, 0, 255), 1)
cv2.line(happy_face, (headshot_size // 2, 0), (headshot_size // 2, headshot_size), (0, 255, 0, 255), 1)

angry_face = np.zeros((headshot_size, headshot_size, 4), dtype=np.uint8)
cv2.circle(angry_face, (headshot_size // 2, headshot_size // 2), headshot_size // 2, (0, 0, 255, 255), 2)
cv2.line(angry_face, (0, headshot_size // 2), (headshot_size, headshot_size // 2), (0, 0, 255, 255), 1)
cv2.line(angry_face, (headshot_size // 2, 0), (headshot_size // 2, headshot_size), (0, 0, 255, 255), 1)

def overlay_image_alpha(img, img_overlay, pos, alpha_mask):
    x, y = pos
    y1, y2 = max(0, y), min(img.shape[0], y + img_overlay.shape[0])
    x1, x2 = max(0, x), min(img.shape[1], x + img_overlay.shape[1])

    y1o, y2o = max(0, -y), min(img_overlay.shape[0], img.shape[0] - y)
    x1o, x2o = max(0, -x), min(img_overlay.shape[1], img.shape[1] - x)

    if y1 >= y2 or x1 >= x2 or y1o >= y2o or x1o >= x2o:
        return

    channels = img.shape[2]
    alpha = alpha_mask[y1o:y2o, x1o:x2o]
    alpha_inv = 1.0 - alpha

    for c in range(channels):
        img[y1:y2, x1:x2, c] = (alpha * img_overlay[y1o:y2o, x1o:x2o, c] +
                                alpha_inv * img[y1:y2, x1:x2, c])

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Decode the QR code present in the frame
    qr_codes = decode(frame)
    for qr_code in qr_codes:
        # Extract the data from the QR code
        qr_data = qr_code.data.decode('utf-8')
        print(f"QR Code Data: {qr_data}")

        # Draw a rectangle around the QR code
        (x, y, w, h) = qr_code.rect
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Display the decoded data on the screen
        cv2.putText(frame, qr_data, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Calculate the center of the QR code
        center_x = x + w // 2
        center_y = y + h // 2

        # If the QR code indicates an "enemy," display the angry face symbol and send message
        if qr_data.lower() == "enemy":
            # Calculate the position to place the angry face
            headshot_x = center_x - headshot_size // 2
            headshot_y = center_y - headshot_size // 2

            # Overlay the angry face on the frame centered on the QR code
            overlay_image_alpha(frame, angry_face, (headshot_x, headshot_y), angry_face[:, :, 3] / 255.0)

            # Send message to ESP32 for enemy detection
            requests.get(ESP32_URL, params={
                'status': 'enemy',
                'x': center_x,
                'y': center_y
            })
            print(f"Enemy detected! Coordinates sent to ESP32: ({center_x}, {center_y})")

        # If the QR code indicates a "friend," display the happy face symbol and send message
        elif qr_data.lower() == "friendly":
            # Calculate the position to place the happy face
            headshot_x = center_x - headshot_size // 2
            headshot_y = center_y - headshot_size // 2

            # Overlay the happy face on the frame centered on the QR code
            overlay_image_alpha(frame, happy_face, (headshot_x, headshot_y), happy_face[:, :, 3] / 255.0)

            # Send message to ESP32 for friend detection
            requests.get(ESP32_URL, params={
                'status': 'friendly',
                'x': center_x,
                'y': center_y
            })
            print(f"Friend detected! Coordinates sent to ESP32: ({center_x}, {center_y})")

    # Display the resulting frame
    cv2.imshow('TURRET', frame)
    
    # Press 'q' to quit the program
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()


Press 'q' to exit
QR Code Data: Friendly
Friend detected! Coordinates sent to ESP32: (343, 206)
QR Code Data: Friendly
Friend detected! Coordinates sent to ESP32: (350, 202)
QR Code Data: Friendly
Friend detected! Coordinates sent to ESP32: (349, 202)
QR Code Data: Friendly
Friend detected! Coordinates sent to ESP32: (349, 204)
QR Code Data: Friendly
Friend detected! Coordinates sent to ESP32: (376, 221)
QR Code Data: Friendly
Friend detected! Coordinates sent to ESP32: (385, 231)
QR Code Data: Friendly
Friend detected! Coordinates sent to ESP32: (386, 230)
QR Code Data: Friendly
Friend detected! Coordinates sent to ESP32: (385, 234)
QR Code Data: Friendly
Friend detected! Coordinates sent to ESP32: (384, 235)
QR Code Data: Friendly
Friend detected! Coordinates sent to ESP32: (384, 235)
QR Code Data: Friendly
Friend detected! Coordinates sent to ESP32: (386, 237)
QR Code Data: Friendly
Friend detected! Coordinates sent to ESP32: (388, 236)
QR Code Data: Friendly
Friend detected! Coordi